<a href="https://colab.research.google.com/github/z4n0/LessonsExtractor/blob/main/LessonsExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%ls /content/drive/MyDrive/lessonsExtractor

10_AN2DL_CS_Localization_WSL_Explanations.mp4       AOS.mp4                exe_es.mp4
9_Semantic_Segmentation_converted.docx              converted_video.mp4    LessonsExtractor.ipynb
9_Semantic_Segmentation_converted.mp4               CSR-20221124.mp4       output.docx
9_Semantic_Segmentation_converted_slides_only.docx  exe_es_converted.docx  slides/
9_Semantic_Segmentation.mp4                         exe_es_converted.mp4


NB: TURN ON THE T4 GPU

In [3]:
!pip install opencv-python

In [4]:
%cd /content/drive/MyDrive/lessonsExtractor/

/content/drive/MyDrive/lessonsExtractor


In [5]:
%ls

10_AN2DL_CS_Localization_WSL_Explanations.mp4       AOS.mp4                exe_es.mp4
9_Semantic_Segmentation_converted.docx              converted_video.mp4    LessonsExtractor.ipynb
9_Semantic_Segmentation_converted.mp4               CSR-20221124.mp4       output.docx
9_Semantic_Segmentation_converted_slides_only.docx  exe_es_converted.docx  slides/
9_Semantic_Segmentation.mp4                         exe_es_converted.mp4


## conversion from mkv to mp4
NB skip this section if you already have a mp4 as input

In [ ]:
!ffmpeg -i prova.mkv

In [ ]:
#!ffmpeg -i prova.mkv -c:v libx264 -c:a aac prova_converted.mp4

reduce quality and bit rate to help the code run faster

In [ ]:
!ffmpeg -i prova.mkv -s 1280x720 -b:v 1M -c:v libx264 -c:a aac prova_converted.mp4

CODE TO REDUCE QUALITY AND BIT RATE AND SELECT A PIECE OF THE MP4 VIDEO

#check if ffmpeg is compatible for hw accelleration

TURN ON THE T4 GPU!!

A typical successful output snippet might start like this:


In [ ]:

Encoder h264_nvenc [NVIDIA NVENC H.264 encoder]:
    General capabilities: delay hardware
    Threading capabilities: none
    Supported pixel formats: nv12 p010le yuv444p nv16 yuv444p16le bgr0 rgb0 ...
    ...


SyntaxError: ignored

In [ ]:
#code to check if you can use accelleration
!ffmpeg -h encoder=h264_nvenc

In [ ]:
#check if the tpu is on
!nvidia-smi

In [ ]:
%ls

converted_video.mp4  CSR-20221124.mp4  lessonsExtractor.ipynb  slides/


#slide extractor

In [6]:
import cv2
import numpy as np
import os

In [ ]:
%ls #to copy paste the name of the video you want to transcribe

In [19]:
input_video_path = '/content/drive/MyDrive/lessonsExtractor/10_AN2DL_CS_Localization_WSL_Explanations.mp4'

# Extract base name and remove extension to form the new output file name
base_name = os.path.basename(input_video_path)
file_name_without_ext = os.path.splitext(base_name)[0]
output_video_path = f"/content/drive/MyDrive/lessonsExtractor/{file_name_without_ext}_converted.mp4"
resolution = "1280x720"
bitrate = "1M"
start_time = "00:00:00"  # Start at 0 minutes

In [9]:
#duration = "00:30:00"  # Duration of 50 minutes, will stop at the end of the video if it's shorter

# Standard FFmpeg command
#!ffmpeg -ss {start_time} -i {input_video_path} -t {duration} -s {resolution} -b:v {bitrate} -c:v libx264 -c:a aac {output_video_path}

# GPU accelerated version
#!ffmpeg -hwaccel cuda -ss {start_time} -i {input_video_path} -s {resolution} -b:v {bitrate} -c:v h264_nvenc -c:a aac {output_video_path}
#!ffmpeg -i {input_video_path} -ss {start_time} -s {resolution} -b:v {bitrate} -c:v h264_nvenc -c:a aac {output_video_path}

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

you should see the new converted_video.mp4 here. let's check!

In [22]:
%ls

10_AN2DL_CS_Localization_WSL_Explanations.mp4       CSR-20221124.mp4       LessonsExtractor.ipynb
9_Semantic_Segmentation_converted.mp4               exe_es_converted.docx  output.docx
9_Semantic_Segmentation_converted_slides_only.docx  exe_es_converted.mp4
AOS.mp4                                             exe_es.mp4


**NB: THE PARAMETER THRESHOLD IN DETECT_SLIDES(....) HAS TO BE FINE TUNED FOR EACH VIDEO**

the higher the value the more selective it becomes

In [23]:
def sum_time(time1, time2):
    # Convert time to seconds
    def time_to_seconds(time_str):
        h, m, s = map(int, time_str.split(':'))
        return h * 3600 + m * 60 + s

    # Convert seconds back to time
    def seconds_to_time(seconds):
        h = seconds // 3600
        m = (seconds % 3600) // 60
        s = seconds % 60
        return f"{h:02d}:{m:02d}:{s:02d}"

    total_seconds = time_to_seconds(time1) + time_to_seconds(time2)
    return seconds_to_time(total_seconds)

In [26]:
def save_frame(frame, timestamp, output_folder='slides'):
    print(sum_time(timestamp, start_time)) ## WHERE sum_time IS DEFINED?
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    filename = f"{output_folder}/slide_{timestamp}.jpg"
    cv2.imwrite(filename, frame)
    print(f"Saved: {filename}")

def frame_difference(prev_frame, curr_frame):
    # Compute the absolute difference between the current frame and the previous frame
    diff = cv2.absdiff(prev_frame, curr_frame)
    non_zero_count = np.count_nonzero(diff)
    return non_zero_count

def milliseconds_to_hh_mm_ss(milliseconds):
    hours = int(milliseconds / 3600000)
    minutes = int((milliseconds % 3600000) / 60000)
    seconds = int((milliseconds % 60000) / 1000)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

#trashold is to be fine tuned for each video
def detect_slides(video_path, threshold=400000, skip_frames=30):
    cap = cv2.VideoCapture(video_path)

    ret, prev_frame = cap.read()
    if not ret:
        print("Failed to read the video")
        return []

    prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_count = 0
    slide_changes = []

    while cap.isOpened():
        ret, curr_frame = cap.read()
        if not ret:
            break

        if frame_count % skip_frames == 0:
            gray_frame = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
            if frame_difference(prev_frame, gray_frame) > threshold:
                timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
                timestamp_readable = milliseconds_to_hh_mm_ss(timestamp_ms)
                adjusted_timestamp = sum_time(timestamp_readable, start_time)  # Adjust the timestamp
                slide_changes.append(adjusted_timestamp)
                save_frame(curr_frame, adjusted_timestamp)  # Use adjusted timestamp
                prev_frame = gray_frame
        frame_count += 1

    cap.release()
    return slide_changes

#input_video_path = output_video_path
slide_changes = detect_slides(input_video_path)
print(f"Slide changes detected at: {slide_changes}")

00:00:01
Saved: slides/slide_00:00:01.jpg
00:00:02
Saved: slides/slide_00:00:02.jpg
00:00:27
Saved: slides/slide_00:00:27.jpg
00:01:31
Saved: slides/slide_00:01:31.jpg
00:02:01
Saved: slides/slide_00:02:01.jpg
00:03:09
Saved: slides/slide_00:03:09.jpg
00:03:14
Saved: slides/slide_00:03:14.jpg
00:03:19
Saved: slides/slide_00:03:19.jpg
00:03:48
Saved: slides/slide_00:03:48.jpg
00:04:30
Saved: slides/slide_00:04:30.jpg
00:05:00
Saved: slides/slide_00:05:00.jpg
00:06:26
Saved: slides/slide_00:06:26.jpg
00:08:54
Saved: slides/slide_00:08:54.jpg
00:09:24
Saved: slides/slide_00:09:24.jpg
00:11:27
Saved: slides/slide_00:11:27.jpg
00:15:21
Saved: slides/slide_00:15:21.jpg
00:15:34
Saved: slides/slide_00:15:34.jpg
00:15:43
Saved: slides/slide_00:15:43.jpg
00:16:03
Saved: slides/slide_00:16:03.jpg
00:17:15
Saved: slides/slide_00:17:15.jpg
00:17:27
Saved: slides/slide_00:17:27.jpg
00:17:28
Saved: slides/slide_00:17:28.jpg
00:19:56
Saved: slides/slide_00:19:56.jpg
00:20:26
Saved: slides/slide_00:20

63

In [27]:
print(f"detected slides {len(slide_changes)}")

detected slides 282


# Audio extraction

In [13]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-2ko7lsfz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-2ko7lsfz
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=4663ff5c9fc3364523046e68ba7421aba3d945423d193d2254aa98812ae4556d
  Stored in directory: /tmp/pip-ephem-wheel-cache-qfxittbp/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dep

In [28]:
import whisper
#if transcription suck change the model size: tiny < base < small < medium < large
#if transcribing from english use .en eg small.en
def transcribe_video(video_path, model_size='medium.en'):
    model = whisper.load_model(model_size)
    result = model.transcribe(video_path)
    return result

In [29]:
def interleave_transcription_with_slides(transcription, slide_changes):
    interleaved_data = []
    current_slide = 0

    for segment in transcription['segments']:
        # Find the next slide change that occurs after this segment starts
        while current_slide < len(slide_changes) and slide_changes[current_slide] <= segment['start']:
            interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
            current_slide += 1

        # Add the transcribed text
        interleaved_data.append({'type': 'transcription', 'timestamp': segment['start'], 'text': segment['text']})

    # Handle any remaining slide changes
    while current_slide < len(slide_changes):
        interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
        current_slide += 1

    return interleaved_data

#pdf or docx creation

In [17]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 3.4 MB/s eta 0:00:00


In [30]:
from docx import Document
from docx.shared import Pt, Inches, Cm
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.section import WD_SECTION_START


def create_document(interleaved_data, video_path, slides_folder='slides'):
    # Extract base name and remove extension to form the DOCX file name
    base_name = os.path.basename(video_path)
    file_name_without_ext = os.path.splitext(base_name)[0]
    output_file = f'{file_name_without_ext}.docx'

    doc = Document()

    style = doc.styles['Normal']
    style.paragraph_format.line_spacing = 1  # Single line spacing
    style.paragraph_format.space_before = Pt(0)
    style.paragraph_format.space_after = Pt(0)

    for item in interleaved_data:
        if item['type'] == 'slide_change':
      # Add slide image
            slide_image_path = f"{slides_folder}/slide_{item['timestamp']}.jpg"
            p = doc.add_paragraph()
            try:
                p.add_run().add_picture(slide_image_path, width=Inches(6))  # Adjust the width as needed
            except Exception as e:
                p.add_run(f"Could not load slide image: {slide_image_path}")
        else:
             # Clean up the transcribed text
            cleaned_text = item['text'].strip().replace('\n', ' ')
            # Add the cleaned text to the document
            p = doc.add_paragraph(cleaned_text)
            # Adjust paragraph formatting as needed
            p.paragraph_format.line_spacing = 1  # Single line spacing
            p.paragraph_format.space_before = Pt(0)
            p.paragraph_format.space_after = Pt(0)

    doc.save(output_file)
    print(f"Document saved as: {output_file}")


def convert_timestamp_to_seconds(timestamp_str):
    parts = timestamp_str.split(':')
    if len(parts) == 3:
        hours, minutes, seconds = map(int, parts)
        return hours * 3600 + minutes * 60 + seconds
    elif len(parts) == 2:
        minutes, seconds = map(int, parts)
        return minutes * 60 + seconds
    else:
        raise ValueError(f"Invalid timestamp format: {timestamp_str}")

def interleave_transcription_with_slides(transcription, slide_changes):
    # Convert slide change timestamps to seconds
    slide_changes_seconds = [convert_timestamp_to_seconds(ts) for ts in slide_changes]

    interleaved_data = []
    current_slide = 0

    for segment in transcription['segments']:
        # Find the next slide change that occurs after this segment starts
        while current_slide < len(slide_changes_seconds) and slide_changes_seconds[current_slide] <= segment['start']:
            interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
            current_slide += 1

        # Add the transcribed text
        interleaved_data.append({'type': 'transcription', 'timestamp': segment['start'], 'text': segment['text']})

    # Handle any remaining slide changes
    while current_slide < len(slide_changes_seconds):
        interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
        current_slide += 1

    return interleaved_data

# Transcribe video
transcription = transcribe_video(input_video_path)

# Interleave transcription with slide changes
interleaved_data = interleave_transcription_with_slides(transcription, slide_changes)

# Create DOCX document
create_document(interleaved_data, output_video_path)

Document saved as: 10_AN2DL_CS_Localization_WSL_Explanations_converted.docx


In [ ]:
%ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
9_Semantic_Segmentation_converted.docx  converted_video.mp4     output.docx
9_Semantic_Segmentation_converted.mp4   CSR-20221124.mp4        slides/
9_Semantic_Segmentation.mp4             LessonsExtractor.ipynb


eliminate the slides images after having generated the docx output

In [31]:
%rm -rf /content/drive/MyDrive/lessonsExtractor/slides

code to put only slides in a docx

In [ ]:
from docx import Document
from docx.shared import Inches
import os

def create_document_with_slides_only(slide_changes, video_path, slides_folder='slides'):
    # Extract base name and remove extension to form the DOCX file name
    base_name = os.path.basename(video_path)
    file_name_without_ext = os.path.splitext(base_name)[0]
    output_file = f'{file_name_without_ext}_slides_only.docx'

    doc = Document()

    for timestamp in slide_changes:
        slide_image_path = f"{slides_folder}/slide_{timestamp}.jpg"
        p = doc.add_paragraph()
        try:
            p.add_run().add_picture(slide_image_path, width=Inches(6))  # Adjust the width as needed
        except Exception as e:
            p.add_run(f"Could not load slide image: {slide_image_path}")

    doc.save(output_file)
    print(f"Document saved as: {output_file}")

# Example usage:
# slide_changes = ['00:00:10', '00:01:15', '00:02:30']  # List of timestamps for slide changes
create_document_with_slides_only(slide_changes, input_video_path)


Document saved as: 9_Semantic_Segmentation_converted_slides_only.docx
